<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1773d0186d375595ee1c8ed583ddcb5ce46d62ca354bfa6f48146013d89c22fc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-23 10:26:01
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 23.81 K (0.17%)
Current PnL: -25.52 L (-16.49%)
CY Booked + Current PnL: -11.23 L (-7.25%)
-------------------
Total profit:  1.64 L
Total loss:  -27.16 L
-------------------
Total Booked + Current PnL: 15.75 L (12.32%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.1%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.15 L (63.7%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.79,-18.66,23.05,0.09,18895.0,-18804.0,81973.0,90.18,40.0,M-SC,2.38,253.0,-1.00,0.58,7.69,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.35,8.20,10.62,19.69,22033.0,15723.0,207465.0,-2.32,53.0,X-MC,2.47,78.0,0.71,1.47,15.55,XY25,NTT,AC
50,MASFIN,398.61,0.81,-2.48,25.15,22.05,24031.0,-2430.0,95550.0,-15.99,53.0,H-SC,6.32,164.0,-0.10,0.68,38.89,XR,ATH,FINANCE
52,NESTLEIND,1377.00,0.06,10.91,9.46,21.40,27848.0,28946.0,294372.0,2.90,56.0,X-LC,5.48,12.0,1.04,2.09,18.27,XY25,NTT,FMCG
39,INDIGOPNTS,1408.00,0.09,-16.08,19.19,0.02,28110.0,-28078.0,146481.0,123.21,43.0,M-SC,12.07,234.0,-1.00,1.04,27.41,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,4.15,-38.94,121.55,35.27,84272.0,-44218.0,69331.0,-53.72,52.0,L-SC,9.70,270.0,-0.52,0.49,69.69,XR,NTT,HOTELS
79,VAIBHAVGBL,521.00,3.67,-24.86,113.96,60.77,156506.0,-45441.0,137334.0,58.13,54.0,H-SC,4.72,159.0,-0.29,0.97,23.68,XR,NTT,JEWELLERY
17,COALINDIA,484.83,3.50,-1.70,21.28,19.23,31475.0,-2552.0,147908.0,15.50,73.0,L-LC,2.12,179.0,-0.08,1.05,19.22,XY25,ATH,MINING
80,VALIANTORG,838.00,2.76,-43.12,206.17,74.15,191858.0,-70547.0,93058.0,-267.19,52.0,H-SC,22.45,149.0,-0.37,0.66,19.29,XR,NTT,CHEMICALS
60,REPCOHOME,880.00,2.08,-13.37,119.12,89.82,295643.0,-38316.0,248189.0,-56.78,50.0,H-SC,5.60,160.0,-0.13,1.76,31.31,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-3.58,-57.03,132.89,0.07,78327.0,-78236.0,58941.0,2060.35,70.0,L-SC,3.62,268.0,-1.00,0.42,46.59,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.84,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,68.0,L-SC,12.17,271.0,-0.05,0.64,74.42,XR,NTT,CERAMICS
41,INFY,1972.00,-1.62,13.16,18.63,34.24,65963.0,41182.0,354070.0,-11.70,67.0,X-LC,5.79,2.0,0.62,2.51,22.53,X40,NTT,IT
67,SONACOMS,806.63,-1.50,-15.92,65.96,39.54,56105.0,-16100.0,85059.0,-34.23,48.0,M-SC,3.84,220.0,-0.29,0.60,19.94,AR,ATH,AUTO
3,ACC,3906.00,-1.25,-26.03,121.91,64.15,214586.0,-61931.0,176020.0,-56.75,37.0,X-SC,3.81,82.0,-0.29,1.25,0.50,XY24,BTT,CEMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.51,-1.13,110.29,107.91,160993.0,-1672.0,145972.0,-19.74,58.0,M-SC,11.88,216.0,-0.01,1.04,8.84,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.09,-16.08,19.19,0.02,28110.0,-28078.0,146481.0,123.21,43.0,M-SC,12.07,234.0,-1.00,1.04,27.41,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.79,-18.66,23.05,0.09,18895.0,-18804.0,81973.0,90.18,40.0,M-SC,2.38,253.0,-1.00,0.58,7.69,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.20,-21.49,68.17,32.03,70557.0,-28333.0,103502.0,-15.03,63.0,H-SC,9.09,148.0,-0.40,0.73,15.93,OX40N,NTT,IT
66,SIS,528.00,-0.63,-23.98,59.52,21.26,50238.0,-26627.0,84405.0,1991.71,51.0,H-SC,4.72,166.0,-0.53,0.60,14.04,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.25,11.56,54.41,72.27,102857.0,19592.0,189040.0,-4.26,76.0,M-LC,3.69,99.0,0.19,1.34,18.45,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.25,11.56,54.41,72.27,102857.0,19592.0,189040.0,-4.26,76.0,M-LC,3.69,99.0,0.19,1.34,18.45,XR,NTT,IT
25,FINCABLES,1641.55,-0.51,-1.13,110.29,107.91,160993.0,-1672.0,145972.0,-19.74,58.0,M-SC,11.88,216.0,-0.01,1.04,8.84,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.79,-3.93,115.18,106.72,136473.0,-4849.0,118487.0,-52.34,40.0,H-SC,4.26,139.0,-0.04,0.84,18.53,AR,ATH,MISC
6,ASIANTILES,137.00,-1.84,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,68.0,L-SC,12.17,271.0,-0.05,0.64,74.42,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.46,-2.73,39.43,35.63,80382.0,-5713.0,203861.0,-16.30,44.0,H-MC,2.40,119.0,-0.07,1.45,13.49,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-0.97,-15.30,48.83,26.07,80745.0,-29861.0,165360.0,-58.78,32.0,X-MC,11.89,56.0,-0.37,1.17,2.99,X40N,ATH,IT
9,BAJAJHFL,181.50,0.23,-17.66,88.83,55.47,177939.0,-42972.0,200314.0,-26.96,33.0,X-MC,15.18,64.0,-0.24,1.42,2.64,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.09,-38.90,121.28,35.20,95348.0,-50052.0,78618.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.84,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.25,-23.70,101.16,53.50,232921.0,-71501.0,230250.0,-63.12,34.0,X-MC,5.38,58.0,-0.31,1.63,2.27,XY24,NTT,FMCG
34,HONAUT,58357.33,-0.24,-17.68,74.10,43.32,99353.0,-28792.0,134080.0,-29.29,34.0,X-SC,5.63,90.0,-0.29,0.95,2.95,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.14,0.01,24.46,24.48,61612.0,33.0,251889.0,-80.87,50.0,X-SC,1.16,86.0,0.00,1.79,23.15,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.75,-2.70,24.50,21.14,54197.0,-6128.0,221211.0,-9.65,48.0,X-MC,1.18,74.0,-0.11,1.57,24.69,XY24,NTT,PAINTS
20,DABUR,735.0,-0.02,-3.50,48.91,43.70,117089.0,-8672.0,239396.0,-11.44,36.0,X-MC,2.09,72.0,-0.07,1.70,11.03,XY24,BTT,FMCG
43,ITC,452.0,0.32,-0.98,11.88,10.78,28269.0,-2362.0,237956.0,-41.15,52.0,X-LC,2.12,5.0,-0.08,1.69,4.20,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.50,6.38,13.53,20.77,33700.0,14929.0,249073.0,-2.93,56.0,X-MC,2.17,62.0,0.44,1.77,13.74,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.25,-26.03,121.91,64.15,214586.0,-61931.0,176020.0,-56.75,37.0,X-SC,3.81,82.0,-0.29,1.25,0.50,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.09,-38.90,121.28,35.20,95348.0,-50052.0,78618.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.84,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,0.16,-19.82,76.49,41.51,161516.0,-52205.0,211160.0,-4.54,41.0,X-MC,7.72,61.0,-0.32,1.50,1.16,XY25,ATH,FMCG
59,RELAXO,1176.00,0.42,-47.66,191.02,52.31,145120.0,-69189.0,75971.0,-44.25,43.0,X-SC,7.25,91.0,-0.48,0.54,1.69,X40N,NTT,FOOTWEAR
8,AWL,485.00,-0.25,-23.70,101.16,53.50,232921.0,-71501.0,230250.0,-63.12,34.0,X-MC,5.38,58.0,-0.31,1.63,2.27,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.14,-29.31,102.07,42.84,46904.0,-19053.0,45953.0,-53.72,46.0,X-SC,34.26,83.0,-0.41,0.33,3.96,XY24,NTT,MISC
59,RELAXO,1176.00,0.42,-47.66,191.02,52.31,145120.0,-69189.0,75971.0,-44.25,43.0,X-SC,7.25,91.0,-0.48,0.54,1.69,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.09,-38.90,121.28,35.20,95348.0,-50052.0,78618.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.84,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,1.04,-1.44,25.89,24.08,32394.0,-1828.0,125122.0,-9.34,46.0,X-SC,6.14,89.0,-0.06,0.89,17.89,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.24,-17.68,74.10,43.32,99353.0,-28792.0,134080.0,-29.29,34.0,X-SC,5.63,90.0,-0.29,0.95,2.95,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.31,-8.58,32.03,20.71,110757.0,-32447.0,345790.0,-21.36,73.0,X-LC,3.65,1.0,-0.29,2.45,15.54,X40,ATH,IT
41,INFY,1972.00,-1.62,13.16,18.63,34.24,65963.0,41182.0,354070.0,-11.70,67.0,X-LC,5.79,2.0,0.62,2.51,22.53,X40,NTT,IT
75,TMPV,600.00,0.60,-25.95,66.07,22.98,110523.0,-58616.0,167282.0,-23.08,52.0,X-LC,6.44,3.0,-0.53,1.19,5.21,XY24,NTT,AUTO
81,VBL,671.64,-0.75,-2.64,39.34,35.67,120977.0,-8326.0,307516.0,-14.21,57.0,X-LC,3.61,4.0,-0.07,2.18,10.85,X40N,ATH,FMCG
43,ITC,452.00,0.32,-0.98,11.88,10.78,28269.0,-2362.0,237956.0,-41.15,52.0,X-LC,2.12,5.0,-0.08,1.69,4.20,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.46,-37.06,111.16,32.89,54013.0,-28616.0,48590.0,-699.86,56.0,L-MC,6.83,259.0,-0.53,0.34,33.81,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-3.58,-57.03,132.89,0.07,78327.0,-78236.0,58941.0,2060.35,70.0,L-SC,3.62,268.0,-1.00,0.42,46.59,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.84,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,68.0,L-SC,12.17,271.0,-0.05,0.64,74.42,XR,NTT,CERAMICS
50,MASFIN,398.61,0.81,-2.48,25.15,22.05,24031.0,-2430.0,95550.0,-15.99,53.0,H-SC,6.32,164.0,-0.10,0.68,38.89,XR,ATH,FINANCE
13,BSOFT,831.70,1.08,-17.51,82.79,50.79,96057.0,-24628.0,116025.0,5.91,74.0,H-SC,7.63,151.0,-0.26,0.82,35.09,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.08,-17.51,82.79,50.79,96057.0,-24628.0,116025.0,5.91,74.0,H-SC,7.63,151.0,-0.26,0.82,35.09,XR,ATH,IT
56,RAJESHEXPO,518.00,-3.58,-57.03,132.89,0.07,78327.0,-78236.0,58941.0,2060.35,70.0,L-SC,3.62,268.0,-1.00,0.42,46.59,OX40N,NTT,JEWELLERY
61,ROUTE,2227.21,0.54,-45.34,209.27,69.05,152213.0,-60331.0,72735.0,-55.87,56.0,H-SC,20.96,140.0,-0.40,0.52,12.17,SR,ATH,IT
6,ASIANTILES,137.00,-1.84,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,68.0,L-SC,12.17,271.0,-0.05,0.64,74.42,XR,NTT,CERAMICS
84,WIPRO,420.00,-0.25,11.56,54.41,72.27,102857.0,19592.0,189040.0,-4.26,76.0,M-LC,3.69,99.0,0.19,1.34,18.45,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.39
1,25,44.83
2,50,76.53


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.94
MC    30.73
LC    24.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.76
X40      23.01
X40N     12.96
XR        9.92
XY25      9.10
AR        8.90
OX40N     7.56
SR        1.01
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.24
H-SC    23.77
X-LC    20.80
M-SC    11.61
X-SC     8.01
H-MC     4.79
L-SC     1.55
M-MC     1.36
M-LC     1.34
H-LC     1.11
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.22,-7.52,42.09
IT,13.15,-13.33,70.32
FINANCE,9.52,-18.78,70.45
MISC,7.28,-26.84,78.72
ELECTRICAL,5.96,-11.80,52.75
PAINTS,5.78,-13.33,30.17
INSURANCE,4.75,-1.33,36.01
PHARMA,4.06,-2.78,35.13
AUTO,2.86,-30.28,75.44


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3154233.0,21
AR,1317473.0,10
XR,1262854.0,13
X40,980613.0,14
X40N,858724.0,9
OX40N,725315.0,10
XY25,359375.0,6
SR,278135.0,2
MH,78675.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3627426.0,25
M-SC,1423497.0,15
X-MC,1410818.0,16
X-LC,794996.0,11
X-SC,791784.0,8
H-MC,404165.0,3
L-SC,242965.0,3
M-LC,102857.0,1
H-LC,77472.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1268327.0      6
           AR         902338.0      5
M-SC       XY24       810255.0      6
H-SC       XR         768782.0      7
X-MC       X40        485233.0      7
           XY24       404207.0      3
           X40N       337829.0      4
H-SC       OX40N      331169.0      4
M-SC       OX40N      315819.0      5
X-SC       X40N       306085.0      3
X-LC       X40        303565.0      5
X-SC       XY24       293884.0      3
H-SC       SR         278135.0      2
X-LC       X40N       214810.0      2
H-MC       AR         212804.0      2
X-SC       X40        191815.0      2
H-MC       XY24       191361.0      1
X-LC       XY24       186199.0      2
X-MC       XY25       183549.0      2
M-SC       XR         172564.0      2
L-SC       XR         164638.0      2
M-SC       AR         124859.0      2
M-LC       XR         102857.0      1
X-LC       XY25        90422.0      2
H-SC       MH          78675.0      1
L-SC       OX40N       78327.0      1
H-LC       AR          77472.0      1
L-MC       XR          54013.0      1
M-MC       XY25        53929.0      1
L-LC       XY25        31475.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
